
# N0 configuration with PML

Trying to add exacty PML.  At the moment debugging.

In [ ]:
import numpy as np

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from scipy.linalg import null_space

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814

In [ ]:
ts = [4.0775e-05, 2e-5, 1e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]
mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [.1,.03, .02]
pml = {'alpha':1, 'R0':rhos[-2]}

In [ ]:
A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs, wl=7.2e-6)


In [ ]:
A.rhos/A.scale

In [ ]:
k_low = A.k0 * A.ns[0] * A.scale
k_low

In [ ]:
outer = 'pcb'
nu = 1

In [ ]:
zero_radial=False
plotlogf(A.determinant, .995*k_low, 1.00001*k_low, -.03,.03, nu, outer, pml,zero_radial,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(13.06 -.003j)

beta1 = newton(A.determinant, guess, args=(nu, outer, pml, zero_radial), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer, pml, zero_radial)))


Get coeffs and K for checking.

In [ ]:
L, R = A.determinant(beta1, return_matrix=True, zero_radial=zero_radial, outer=outer, nu=nu, pml=pml)

In [ ]:
Aa, B = L[..., 0, 0], L[..., 0, 1]
C, D = L[..., 1, 0], L[..., 1, 1]


In [ ]:
np.abs(Aa * D - B * C)**2

In [ ]:
L @ np.array([L[0,1], -L[0,0]])

# Bragg Exact Fields

In [ ]:
Fs = A.all_fields(beta1, nu, outer, pml, zero_radial)

In [ ]:
Draw(1e1*Fs['Ez'], A.mesh)

In [ ]:
Draw(Fs['Ephi'], A.mesh)

In [ ]:
Draw(Fs['Er'], A.mesh)

In [ ]:
Draw(Fs['Etv'].imag, A.mesh)

In [ ]:
Draw(1e1*Fs['Hz'], A.mesh)

In [ ]:
Draw(Fs['Hphi'].imag, A.mesh)